In [1]:
import datarobot as dr
from PredictionHelpers import * 
import pandas as pd
import io

In [2]:
stream = io.StringIO()
df = pd.read_csv("scoring_data.csv")
df.head()

,Unnamed: 0,Unnamed: 0.1,Policy_ID,Eff_Dt,IncurredClaims,Exposure,Zipcode,VehicleCostNew,VehicleMake,VehicleModel,...,Zipcode_Aged_30_39,Zipcode_Aged_40_44,Zipcode_Aged_45_49,Zipcode_Aged_50_59,Zipcode_Aged_60,Zipcode_PersonsPerHousehold,Zipcode_annualMileage,Zipcode_VehiclesPerHousehold,Zipcode_CommuteViaCar,DistributionChannel
0,0,0,110001,2017-05-01,14448.33,1.00,33672,40000.0,MERCEDES-BENZ,C200,...,0.078153,0.087062,0.246694,0.125386,0.160068,2.039301,2668.979414,0.908916,0.40,Insurance Broker
1,1,1,110002,2017-04-29,0.00,1.00,29706,37000.0,FORD,Ranger,...,0.095329,0.104835,0.184922,0.162141,0.244797,2.445253,4005.068973,1.412275,0.60,Insurance Agency
2,2,2,110003,2017-08-18,18549.37,1.00,35555,39000.0,FORD,Ranger,...,0.094120,0.097575,0.202296,0.176526,0.168268,2.616979,5751.105274,1.196203,0.59,Insurance Agency
3,3,3,110004,2017-08-02,2152.92,0.78,26155,107000.0,TOYOTA,Camry,...,0.080600,0.092339,0.242043,0.160232,0.197870,1.999606,2962.848071,1.092110,0.51,Vehicle Dealership
4,4,4,110005,2017-03-24,0.00,0.43,40363,22000.0,HONDA,Accord,...,0.086897,0.100755,0.231012,0.126033,0.202581,2.186630,3224.389464,0.806077,0.37,Vehicle Dealership


In [3]:
df.head(100).to_csv(stream)

In [4]:
predictions = make_datarobot_deployment_predictions(stream.getvalue(), "5f2029666ac9eb0866055e48")

In [5]:
predictions

{'data': [{'deploymentApprovalStatus': 'PENDING',
   'passthroughValues': {'Policy_ID': '110001'},
   'rowId': 0,
   'predictionValues': [{'value': 1445.2172347176, 'label': 'IncurredClaims'}],
   'prediction': 1445.2172347176,
   'predictionExplanations': [{'featureValue': 20.0401544016,
     'strength': -2623.0209612617,
     'feature': 'VehicleAge',
     'qualitativeStrength': '---',
     'label': 'IncurredClaims'},
    {'featureValue': 0.144983768,
     'strength': 607.0333175473,
     'feature': 'Zipcode_Aged_25_29',
     'qualitativeStrength': '++',
     'label': 'IncurredClaims'},
    {'featureValue': 'MERCEDES-BENZ',
     'strength': -449.901387957,
     'feature': 'VehicleMake',
     'qualitativeStrength': '--',
     'label': 'IncurredClaims'}]},
  {'deploymentApprovalStatus': 'PENDING',
   'passthroughValues': {'Policy_ID': '110002'},
   'rowId': 1,
   'predictionValues': [{'value': 1368.1318047115, 'label': 'IncurredClaims'}],
   'prediction': 1368.1318047115,
   'prediction

In [6]:
def flatten_json(y):
    out = {}
    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[str(name[:-1])] = str(x)
    flatten(y)
    return out

In [7]:
records = map(flatten_json, predictions["data"])

In [8]:
pd.DataFrame(records)

,deploymentApprovalStatus,passthroughValues_Policy_ID,rowId,predictionValues_0_value,predictionValues_0_label,prediction,predictionExplanations_0_featureValue,predictionExplanations_0_strength,predictionExplanations_0_feature,predictionExplanations_0_qualitativeStrength,...,predictionExplanations_1_featureValue,predictionExplanations_1_strength,predictionExplanations_1_feature,predictionExplanations_1_qualitativeStrength,predictionExplanations_1_label,predictionExplanations_2_featureValue,predictionExplanations_2_strength,predictionExplanations_2_feature,predictionExplanations_2_qualitativeStrength,predictionExplanations_2_label
0,PENDING,110001,0,1445.2172347176,IncurredClaims,1445.2172347176,20.0401544016,-2623.0209612617,VehicleAge,---,...,0.144983768,607.0333175473,Zipcode_Aged_25_29,++,IncurredClaims,MERCEDES-BENZ,-449.901387957,VehicleMake,--,IncurredClaims
1,PENDING,110002,1,1368.1318047115,IncurredClaims,1368.1318047115,37000.0,-729.4335692167,VehicleCostNew,---,...,0.116033871,-650.9020070587,Zipcode_Aged_18_24,--,IncurredClaims,None,-614.7692235122,DriverAge,--,IncurredClaims
2,PENDING,110003,2,4746.2840476958,IncurredClaims,4746.2840476958,2,774.7589786134,CustomerTenure,++,...,0.176525712,621.4425642248,Zipcode_Aged_50_59,++,IncurredClaims,0.202295888,-612.7415406022,Zipcode_Aged_45_49,--,IncurredClaims
3,PENDING,110004,3,4018.2140006078,IncurredClaims,4018.2140006078,0.100810844,-1337.3600236593,Zipcode_Aged_18_24,--,...,107000.0,988.3974177607,VehicleCostNew,++,IncurredClaims,6.1196501211,906.8858400832,VehicleAge,++,IncurredClaims
4,PENDING,110005,4,187.1289530628,IncurredClaims,187.1289530628,22.5711551798,-606.0702215203,VehicleAge,---,...,10,-242.2773949126,CustomerTenure,--,IncurredClaims,0.43,-215.4158878281,Exposure,--,IncurredClaims
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,PENDING,110096,95,769.1630747052,IncurredClaims,769.1630747052,21.3056547907,-3917.9138883408,VehicleAge,---,...,6742.951184,-1014.9958504641,Zipcode_annualMileage,--,IncurredClaims,0.173727065,-218.1731956368,Zipcode_Aged_45_49,-,IncurredClaims
96,PENDING,110097,96,1202.1958846415,IncurredClaims,1202.1958846415,0.42,-1445.4974327237,Exposure,---,...,45000.0,228.5425868046,VehicleCostNew,++,IncurredClaims,0.196951997,-197.2510299074,Zipcode_Aged_60,--,IncurredClaims
97,PENDING,110098,97,6433.7008194189,IncurredClaims,6433.7008194189,3.5886493428,1780.8354074409,VehicleAge,+++,...,135000.0,915.6070161599,VehicleCostNew,++,IncurredClaims,7,-599.7547120602,CustomerTenure,--,IncurredClaims
98,PENDING,110099,98,7167.8176064097,IncurredClaims,7167.8176064097,165000.0,2082.9438460317,VehicleCostNew,+++,...,2.3231489537,1831.6991211985,VehicleAge,+++,IncurredClaims,Vehicle Dealership,575.4643343541,DistributionChannel,++,IncurredClaims
